In [1]:
from google.colab import drive
drive.mount('drive/')

Mounted at drive/


In [2]:
!pip install flask-ngrok

# Make HTML File

In [4]:
index = '''
<!doctype html>
<title>Skin Predicition</title>
<h2>Select a image file to upload and predict ...</h2>

<form method="post" action="/" enctype="multipart/form-data">
    <dl>
      <p>
        <input type="file" name="file" autocomplete="off" required onchange="loadFile(event)">
      </p>
    </dl>
    
    <p>
      <img id="output"/>
    </p>
    
    <p>  
      <input type="submit" value="Predict">
    </p>
    
  </form>     

<p>
	{% with messages = get_flashed_messages() %}
	  {% if messages %}        
        This skin probably {{ messages[0] }}%
        {{ messages[1] }} type.
        <br>
	  {% endif %}    
	{% endwith %}
</p>

<script>
  var loadFile = function(event) {
    var output = document.getElementById('output');
    output.src = URL.createObjectURL(event.target.files[0]);
    output.onload = function() {
      URL.revokeObjectURL(output.src) // free memory
    }
  };
</script>
'''
file = open("/content/drive/MyDrive/Colabs/Bangkit/tes/templates/index.html","w")
file.write(index)
file.close()

# Deployment with Flask & ngrok

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from flask import Flask, render_template, request, redirect, flash, url_for
import urllib.request
from werkzeug.utils import secure_filename
import os
from flask_ngrok import run_with_ngrok


names = ["Dry", "Combination", "Normal", "Oily", "Sensitive"]


def pred_skin(filename):    
    model = load_model('/content/drive/MyDrive/Colabs/Bangkit/tes/skincaremobilenet.h5')        
    img = image.load_img('/tmp/'+filename, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    label = classes.argmax()
    labelName = names[label]
    print(classes.shape)
    print('This skin probably {:.2f}% {} type'.format(classes[0][label]*100, labelName))     
    return round(classes[0][label]*100,2), labelName


UPLOAD_FOLDER = '/tmp'
app = Flask(__name__,template_folder='/content/drive/MyDrive/Colabs/Bangkit/tes/templates')
run_with_ngrok(app)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/', methods=['POST'])
def submit_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No file selected for uploading')
            return redirect(request.url)
        if file:
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'],filename))            
            acc, label, = pred_skin(filename)            
            flash(acc)
            flash(label)                        
            return redirect('/')


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d0dc6b242294.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Jun/2021 09:39:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2021 09:39:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2021 09:39:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Jun/2021 09:40:13] "POST / HTTP/1.1" 302 -


(1, 5)
This skin probably 96.35% Sensitive type


127.0.0.1 - - [06/Jun/2021 09:40:13] "GET / HTTP/1.1" 200 -
